In [5]:
!pip install -q tensorflow==2.12.0 statsmodels prophet shap scikit-learn matplotlib pandas nbformat



ERROR: Could not find a version that satisfies the requirement tensorflow==2.12.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.12.0


In [6]:
# Standard imports
import os, warnings, math
warnings.filterwarnings('ignore')
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import statsmodels.api as sm
print('TensorFlow version:', tf.__version__)


TensorFlow version: 2.19.0


## 1. Dataset: UCI Household Power Consumption

This notebook downloads the dataset from the UCI archive, parses dates, converts columns to numeric, and resamples to hourly frequency.


In [7]:

# Download & load (Colab: runs wget). If downloading fails, upload the txt file manually.
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip'
zip_path = '/content/household_power_consumption.zip'
csv_path = '/content/household_power_consumption.txt'

if not os.path.exists(csv_path):
    try:
        print('Attempting to download dataset...')
        !wget -q -O {zip_path} {data_url}
        !unzip -o {zip_path} -d /content
        print('Downloaded and unzipped dataset to /content')
    except Exception as e:
        print('Automatic download failed. Please upload the dataset file household_power_consumption.txt to /content or to Colab files. Error:', e)

# Read the dataset
df = pd.read_csv(csv_path, sep=';', low_memory=False, na_values=['?'])
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True, errors='coerce')
df = df.drop(columns=['Date','Time']).set_index('Datetime').apply(pd.to_numeric, errors='coerce')
df = df.sort_index().resample('H').mean()  # resample to hourly
print('Loaded dataset. Shape (after hourly resample):', df.shape)
display(df.head())


Attempting to download dataset...
Archive:  /content/household_power_consumption.zip
  inflating: /content/household_power_consumption.txt  
Downloaded and unzipped dataset to /content
Loaded dataset. Shape (after hourly resample): (34589, 7)


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,,,,,
2006-12-16 17:00:00,4.222889,0.229000,234.643889,18.100000,0.0,0.527778,16.861111
2006-12-16 18:00:00,3.632200,0.080033,234.580167,15.600000,0.0,6.716667,16.866667
2006-12-16 19:00:00,3.400233,0.085233,233.232500,14.503333,0.0,1.433333,16.683333
2006-12-16 20:00:00,3.268567,0.075100,234.071500,13.916667,0.0,0.000000,16.783333
2006-12-16 21:00:00,3.056467,0.076667,237.158667,13.046667,0.0,0.416667,17.216667


## 2. Preprocessing & Feature Engineering
- Handle missing values
- Create lag features and rolling statistics
- Select at least 3 input features + 1 target


In [8]:

def create_lagged_features(df, features, n_lags=48, rolling_windows=[24,72], dropna=True):
    X = df.copy()
    for col in features:
        for lag in range(1, n_lags+1):
            X[f'{col}_lag_{lag}'] = X[col].shift(lag)
        for w in rolling_windows:
            X[f'{col}_roll_mean_{w}'] = X[col].rolling(window=w).mean().shift(1)
            X[f'{col}_roll_std_{w}'] = X[col].rolling(window=w).std().shift(1)
    if dropna:
        X = X.dropna()
    return X

# Choose features (3 inputs + target). We'll use these commonly present columns:
# 'Global_active_power' (target), 'Voltage', 'Global_intensity', 'Sub_metering_1' as extra features
available = df.columns.tolist()
print('Available columns:', available)

target_col = 'Global_active_power' if 'Global_active_power' in df.columns else df.columns[0]
input_features = [c for c in ['Voltage', 'Global_intensity', 'Sub_metering_1'] if c in df.columns]
# Ensure at least 3 features; if not found, pick first numeric columns
if len(input_features) < 3:
    for c in df.columns:
        if c not in input_features and c != target_col:
            input_features.append(c)
        if len(input_features) >= 3:
            break

print('Target:', target_col)
print('Input features (3):', input_features[:3])

# Subset and create features (include target in DF for lagging)
use_cols = input_features[:3] + [target_col]
sub = df[use_cols].copy()
sub = sub.fillna(method='ffill').fillna(method='bfill')  # simple imputation
processed = create_lagged_features(sub, [target_col] + input_features[:3], n_lags=48, rolling_windows=[24,72])
print('Processed shape:', processed.shape)
processed.head()


Available columns: ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
Target: Global_active_power
Input features (3): ['Voltage', 'Global_intensity', 'Sub_metering_1']
Processed shape: (34517, 212)


,Voltage,Global_intensity,Sub_metering_1,Global_active_power,Global_active_power_lag_1,Global_active_power_lag_2,Global_active_power_lag_3,Global_active_power_lag_4,Global_active_power_lag_5,Global_active_power_lag_6,...,Sub_metering_1_lag_43,Sub_metering_1_lag_44,Sub_metering_1_lag_45,Sub_metering_1_lag_46,Sub_metering_1_lag_47,Sub_metering_1_lag_48,Sub_metering_1_roll_mean_24,Sub_metering_1_roll_std_24,Sub_metering_1_roll_mean_72,Sub_metering_1_roll_std_72
Datetime,,,,,,,,,,,,,,,,,,,,,
2006-12-19 17:00:00,238.606667,8.843333,0.0,2.111500,1.372133,0.421367,0.302133,0.300467,0.430667,2.321633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.582639,2.092877,0.91088,2.710686
2006-12-19 18:00:00,237.640500,9.256667,0.0,2.204700,2.111500,1.372133,0.421367,0.302133,0.300467,0.430667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.582639,2.092877,0.91088,2.710686
2006-12-19 19:00:00,241.080000,7.603333,0.0,1.842100,2.204700,2.111500,1.372133,0.421367,0.302133,0.300467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.582639,2.092877,0.91088,2.710686
2006-12-19 20:00:00,241.328167,12.480000,0.0,2.940533,1.842100,2.204700,2.111500,1.372133,0.421367,0.302133,...,0.0,0.0,0.0,0.0,0.0,0.0,0.582639,2.092877,0.91088,2.710686
2006-12-19 21:00:00,240.870000,6.343333,0.0,1.442867,2.940533,1.842100,2.204700,2.111500,1.372133,0.421367,...,0.0,0.0,0.0,0.0,0.0,0.0,0.582639,2.092877,0.91088,2.710686


## 3. Train/Validation Split, Scaling, and Sequence Creation
We create sequences of length `seq_len` and predict `horizon` steps ahead (multi-step forecasting).

In [9]:

# Time-based train/validation split
train_frac = 0.75
n_train = int(len(processed) * train_frac)
train_df = processed.iloc[:n_train]
val_df = processed.iloc[n_train:]

feature_cols = [c for c in processed.columns if c != target_col]

# Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(train_df[feature_cols])
X_val_scaled = scaler.transform(val_df[feature_cols])
y_train = train_df[target_col].values
y_val = val_df[target_col].values

# Sequence creation
seq_len = 48
horizon = 6

def make_sequences(X, y, seq_len=48, horizon=6):
    Xs, ys = [], []
    for i in range(len(X) - seq_len - horizon + 1):
        Xs.append(X[i:(i+seq_len), :])
        ys.append(y[i+seq_len:i+seq_len+horizon])
    return np.array(Xs), np.array(ys)

X_tr_seq, y_tr_seq = make_sequences(X_train_scaled, y_train, seq_len=seq_len, horizon=horizon)
X_val_seq, y_val_seq = make_sequences(X_val_scaled, y_val, seq_len=seq_len, horizon=horizon)

print('X_tr_seq shape:', X_tr_seq.shape)
print('y_tr_seq shape:', y_tr_seq.shape)


X_tr_seq shape: (25834, 48, 211)
y_tr_seq shape: (25834, 6)


## 4. Baseline LSTM (many-to-many)
A simple baseline LSTM that maps input sequences to horizon outputs. Use GPU in Colab for training.

In [10]:

def build_baseline_lstm(input_shape, units=128, horizon=6):
    inp = keras.Input(shape=input_shape)
    x = layers.LSTM(units)(inp)
    x = layers.Dense(units, activation='relu')(x)
    out = layers.Dense(horizon)(x)
    model = keras.Model(inp, out)
    model.compile(optimizer='adam', loss='mse', metrics=[keras.metrics.RootMeanSquaredError(), keras.metrics.MeanAbsoluteError()])
    return model

baseline = build_baseline_lstm((X_tr_seq.shape[1], X_tr_seq.shape[2]), units=128, horizon=y_tr_seq.shape[1])
baseline.summary()

# Training scaffold (comment/uncomment to run)
# history_baseline = baseline.fit(X_tr_seq, y_tr_seq, validation_data=(X_val_seq, y_val_seq),
#                                epochs=30, batch_size=64, callbacks=[keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)])


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 211)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       174,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 191,366 (747.52 KB)

 Trainable params: 191,366 (747.52 KB)

 Non-trainable params: 0 (0.00 B)

## 5. Seq2Seq Encoder-Decoder with Bahdanau Attention
We implement a Bahdanau-style additive attention and build a training-time model that uses teacher forcing (previous ground truth as decoder input). For inference we'll provide a separate decoding loop that extracts attention weights.

In [11]:

# Bahdanau attention layer
class BahdanauAttention(layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = layers.Dense(units)
        self.W2 = layers.Dense(units)
        self.V = layers.Dense(1)
    def call(self, query, values):
        # query: (batch, hidden), values: (batch, time, hidden)
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(query_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)  # (batch, time, 1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)  # (batch, hidden)
        return context_vector, tf.squeeze(attention_weights, -1)  # return weights (batch, time)

def build_seq2seq_attention(input_shape, encoder_units=128, decoder_units=128, horizon=6):
    encoder_inputs = keras.Input(shape=input_shape, name='encoder_inputs')
    encoder_lstm = layers.LSTM(encoder_units, return_sequences=True, return_state=True, name='encoder_lstm')
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

    decoder_inputs = layers.Input(shape=(horizon, 1), name='decoder_inputs')
    attention = BahdanauAttention(decoder_units)
    decoder_lstm_cell = layers.LSTMCell(decoder_units, name='decoder_lstm_cell')
    all_outputs = []
    decoder_state_h, decoder_state_c = state_h, state_c
    x = decoder_inputs
    for t in range(horizon):
        x_t = layers.Lambda(lambda x: x[:, t:t+1, :])(x)
        x_t = layers.TimeDistributed(layers.Dense(decoder_units))(x_t)
        x_t = layers.Reshape((decoder_units,))(x_t)
        context_vector, attn_weights = attention(decoder_state_h, encoder_outputs)
        decoder_input_and_context = layers.Concatenate()([x_t, context_vector])
        output, [decoder_state_h, decoder_state_c] = decoder_lstm_cell(decoder_input_and_context, states=[decoder_state_h, decoder_state_c])
        out = layers.Dense(1)(output)
        out = layers.Reshape((1,1))(out)
        all_outputs.append(out)
    decoder_outputs = layers.Concatenate(axis=1)(all_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='mse', metrics=[keras.metrics.RootMeanSquaredError(), keras.metrics.MeanAbsoluteError()])
    return model

input_shape = (X_tr_seq.shape[1], X_tr_seq.shape[2])
seq2seq = build_seq2seq_attention(input_shape, encoder_units=128, decoder_units=128, horizon=y_tr_seq.shape[1])
seq2seq.summary()

# Training scaffold: prepare decoder inputs (teacher forcing) as shifted ground truth
# decoder_input_data = np.zeros((y_tr_seq.shape[0], y_tr_seq.shape[1], 1))
# decoder_input_data[:,1:,:] = y_tr_seq[:,:-1].reshape(y_tr_seq.shape[0], y_tr_seq.shape[1]-1, 1)
# decoder_input_data[:,0,:] = y_tr_seq[:,0]  # or zeros
# history_seq2seq = seq2seq.fit([X_tr_seq, decoder_input_data], y_tr_seq.reshape(y_tr_seq.shape[0], y_tr_seq.shape[1], 1),
#                               validation_data=([X_val_seq, decoder_input_val], y_val_seq.reshape(y_val_seq.shape[0], y_val_seq.shape[1], 1)),
#                               epochs=40, batch_size=64, callbacks=[keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)])


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_inputs      │ (None, 6, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1, 1)      │          0 │ decoder_inputs[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_inputs      │ (None, 48, 211)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 1, 128)    │        256 │ lambda[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 48, 128), │    174,080 │ encoder_inputs[0… │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 128)       │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bahdanau_attention  │ [(None, 128),     │     33,153 │ encoder_lstm[0][… │
│ (BahdanauAttention) │ (None, 48)]       │            │ encoder_lstm[0][… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ encoder_lstm[0][… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ encoder_lstm[0][… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ encoder_lstm[0][… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ encoder_lstm[0][… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ reshape[0][0],    │
│ (Concatenate)       │                   │            │ bahdanau_attenti… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 1, 1)      │          0 │ decoder_inputs[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm_cell   │ [(None, 128),     │    197,120 │ concatenate[0][0… │
│ (LSTMCell)          │ (None, 128),      │            │ encoder_lstm[0][… │
│                     │ (None, 128)]      │            │ encoder_lstm[0][… │
│                     │                   │            │ concatenate_1[0]… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ concatenate_2[0]… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ concatenate_3[0]… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ decoder_lstm_cel… │
│                     │                   │            │ concatenate_4[0]… │
│                     │                   │            │ decoder_lstm_cel

 Total params: 406,663 (1.55 MB)

 Trainable params: 406,663 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

### 5.1 Inference decoder to extract attention weights
Below we build an inference routine that runs the encoder on input, then decodes autoregressively while collecting attention weights at each timestep. This is executed in notebook after training to visualize attention.

In [12]:

# Build encoder inference model to return encoder outputs and states
def build_inference_models(seq2seq_model, encoder_units=128, decoder_units=128, horizon=6):
    # Recreate encoder inputs & outputs by name lookup
    encoder_inputs = seq2seq_model.get_layer('encoder_inputs').input
    encoder_lstm = seq2seq_model.get_layer('encoder_lstm')
    encoder_outputs, state_h_enc, state_c_enc = encoder_lstm(encoder_inputs)
    encoder_model = keras.Model(encoder_inputs, [encoder_outputs, state_h_enc, state_c_enc])

    # Decoder inputs for inference
    decoder_state_input_h = keras.Input(shape=(decoder_units,), name='dec_state_h')
    decoder_state_input_c = keras.Input(shape=(decoder_units,), name='dec_state_c')
    encoder_outputs_input = keras.Input(shape=(input_shape[0], encoder_units), name='enc_outputs_input')
    # We'll reuse attention and decoder cell layers from the trained model
    attention = None
    for layer in seq2seq_model.layers:
        if isinstance(layer, BahdanauAttention):
            attention = layer
            break
    if attention is None:
        raise ValueError('Attention layer not found in model.')
    decoder_lstm_cell = None
    for layer in seq2seq_model.layers:
        if 'decoder_lstm_cell' in layer.name:
            decoder_lstm_cell = layer
            break
    if decoder_lstm_cell is None:
        # fallback: create a new LSTMCell (weights won't match)
        decoder_lstm_cell = layers.LSTMCell(decoder_units)

    # Decoder loop for one timestep (we will call in Python loop)
    decoder_input_single = keras.Input(shape=(1,), name='dec_input_single')  # scalar input
    x = layers.Dense(decoder_units)(decoder_input_single)
    # compute context
    context_vector, attn_weights = attention(decoder_state_input_h, encoder_outputs_input)
    decoder_input_and_context = layers.Concatenate()([x, context_vector])
    decoder_output, state_h, state_c = decoder_lstm_cell(decoder_input_and_context, states=[decoder_state_input_h, decoder_state_input_c])
    output = layers.Dense(1)(decoder_output)

    decoder_model = keras.Model([decoder_input_single, encoder_outputs_input, decoder_state_input_h, decoder_state_input_c],
                                [output, state_h, state_c, attn_weights])
    return encoder_model, decoder_model

# The build_inference_models function is a template; it will be used after training to extract attention weights.


## 6. Classical Baseline: SARIMAX
A simple SARIMAX fit on the training target series as an univariate benchmark.

In [13]:

# SARIMAX training & forecast (example)
ts_train = train_df[target_col].dropna()
ts_val = val_df[target_col].dropna()

# Fit a simple SARIMAX (tune orders for better results)
arima_model = sm.tsa.SARIMAX(ts_train, order=(1,1,1), seasonal_order=(0,0,0,0),
                            enforce_stationarity=False, enforce_invertibility=False)
# fitting may take time; run in Colab runtime
# arima_res = arima_model.fit(disp=False)
# arima_forecast = arima_res.get_forecast(steps=len(ts_val))
# arima_pred = arima_forecast.predicted_mean.values[:len(ts_val)]
# print('ARIMA RMSE:', np.sqrt(mean_squared_error(ts_val[:len(arima_pred)], arima_pred)))
print('SARIMAX code scaffolded; run fitting in Colab to produce results.')


SARIMAX code scaffolded; run fitting in Colab to produce results.


## 7. Walk-forward evaluation utilities
Includes: a Keras wrapper to use in the rolling-origin evaluation and a simple sklearn-like wrapper for classical models.

In [14]:

# Simple Keras wrapper to fit and predict in the sklearn style for one-step forecasting per split
class KerasSklearnWrapper:
    def __init__(self, build_fn, epochs=10, batch_size=64, **kwargs):
        self.build_fn = build_fn
        self.epochs = epochs
        self.batch_size = batch_size
        self.model_kwargs = kwargs
        self.model = None
    def fit(self, X, y, **fit_kwargs):
        self.model = self.build_fn(**self.model_kwargs)
        # if y has shape (n,h) and model expects (n,h,1) adapt accordingly
        if len(y.shape) == 2 and self.model.output_shape[-1] == 1:
            y_to_fit = y.reshape(y.shape[0], y.shape[1], 1)
        else:
            y_to_fit = y
        self.model.fit(X, y_to_fit, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self
    def predict(self, X):
        preds = self.model.predict(X)
        if preds.ndim == 3 and preds.shape[-1] == 1:
            preds = preds.reshape(preds.shape[0], -1)
        return preds

def walk_forward_evaluate_model(wrapper_cls, X, y, train_window, step=24, **wrapper_kwargs):
    preds = []
    truths = []
    for start in range(0, len(X) - train_window - 1, step):
        X_train = X[start:start+train_window]
        y_train = y[start:start+train_window]
        X_test = X[start+train_window:start+train_window+1]
        y_test = y[start+train_window:start+train_window+1]
        w = wrapper_cls(**wrapper_kwargs)
        w.fit(X_train, y_train)
        p = w.predict(X_test)
        preds.append(p)
        truths.append(y_test)
    preds = np.vstack(preds)
    truths = np.vstack(truths)
    return preds, truths

print('Walk-forward utilities defined. Example: use train_window=200 and wrapper_cls=KerasSklearnWrapper with build_fn=build_baseline_lstm')


Walk-forward utilities defined. Example: use train_window=200 and wrapper_cls=KerasSklearnWrapper with build_fn=build_baseline_lstm


## 8. Attention Heatmap Visualization (once attention weights available)
The following cell visualizes attention weights for a selected validation sample as a heatmap.

In [15]:

# Example plotting function (expects attn_weights shape (horizon, seq_len) for a single sample)
import matplotlib.pyplot as plt
import seaborn as sns

def plot_attention_heatmap(attn_weights, seq_timestamps=None, figsize=(10,4), title='Attention weights'):
    # attn_weights: (horizon, seq_len)
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(attn_weights, cmap='viridis', xticklabels=seq_timestamps, yticklabels=[f'h{t+1}' for t in range(attn_weights.shape[0])])
    ax.set_xlabel('Encoder time steps (older -> newer)')
    ax.set_ylabel('Decoder timestep (forecast horizon)')
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

# NOTE: seaborn may need installation in Colab: !pip install seaborn
print('Plotting function defined. After running inference and collecting attention weights, call plot_attention_heatmap(attn_weights).')


Plotting function defined. After running inference and collecting attention weights, call plot_attention_heatmap(attn_weights).


## 9. Evaluate and compare models
Compute RMSE/MAE/MAPE and assemble a results table.

In [16]:

from sklearn.metrics import mean_absolute_percentage_error as mape

def eval_forecasts(y_true, y_pred):
    # Accept 1d arrays
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape_val = mape(y_true, y_pred)
    return {'rmse': rmse, 'mae': mae, 'mape': mape_val}

# Example: after computing preds for seq2seq and baseline, build a DataFrame
# results = {
#   'model': ['baseline_lstm', 'seq2seq_attention', 'sarimax'],
#   'rmse': [rmse_b, rmse_s, rmse_a],
#   'mae': [mae_b, mae_s, mae_a],
#   'mape': [mape_b, mape_s, mape_a]
# }
# results_df = pd.DataFrame(results).set_index('model')
# display(results_df)
print('Evaluation utilities ready.')


Evaluation utilities ready.
